In [12]:
import numpy as np
import gmsh
import pygmsh
import sys
resolution = 100
sk = 50000
L = 50000
lx = 2000
lz = 5000
ho = 500

# MESH
geometry = pygmsh.geo.Geometry()
model = geometry.__enter__()

    # points of "measurements"
n = 21
x = np.linspace(-2000, 2000, n)
s = []
for i in range(n):
    s.append(model.add_point((x[i], 1, 0), mesh_size=resolution/10))
for i in range(n):
    s.append(model.add_point((x[n-i-1], 2, 0),mesh_size=resolution/10))
        
s_lines = [model.add_line(s[i], s[i+1])
           for i in range(-1, len(s)-1)]
s_loop = model.add_curve_loop(s_lines)
    ##########
    
    # I inclueded this retangle to have inprove the resolution only
v = []
    
for i in range(n):
    v.append(model.add_point((x[i], -1, 0), mesh_size=resolution/10))
for i in range(n):
    v.append(model.add_point((x[n-i-1], -ho, 0),mesh_size=resolution/10))
           
v_lines = [model.add_line(v[i], v[i+1])
           for i in range(-1, len(v)-1)]
v_loop = model.add_curve_loop(v_lines)
    ##########
    
# earth
points = []
points.append(model.add_point((-L, 0, 0),mesh_size=sk))
for i in range(n):
    points.append(model.add_point((x[i], 0, 0),mesh_size=sk))
points.append(model.add_point((L, 0, 0),mesh_size=sk))
points.append(model.add_point((L, L, 0),mesh_size=sk))
points.append(model.add_point((-L, L, 0),mesh_size=sk))

channel_lines = [model.add_line(points[i], points[i+1])
                 for i in range(-1, len(points)-1)]

# air
points2 = [model.add_point((-1.1*L, -4*L, 0),mesh_size=L),
          model.add_point((1.1*L, -4*L, 0),mesh_size=L),
          model.add_point((1.1*L, 1.1*L, 0),mesh_size=L),
          model.add_point((-1.1*L, 1.1*L, 0),mesh_size=L)]

channel_lines2 = [model.add_line(points2[i], points2[i+1])
                 for i in range(-1, len(points2)-1)]

# body
points3 = [model.add_point((-lx/2, ho, 0), mesh_size=resolution),
          model.add_point((lx/2, ho, 0), mesh_size=resolution),
          model.add_point((lx/2, ho+lz, 0), mesh_size=resolution),
          model.add_point((-lx/2, ho+lz, 0), mesh_size=resolution)]

channel_lines3 = [model.add_line(points3[i], points3[i+1])
                 for i in range(-1, len(points3)-1)]

# Create a line loop and plane surface for meshing
channel_loop = model.add_curve_loop(channel_lines)

channel_loop2 = model.add_curve_loop(channel_lines2)

channel_loop3 = model.add_curve_loop(channel_lines3)

plane_surface = model.add_plane_surface(channel_loop,
                                        holes=[channel_loop3, s_loop])

plane_surface2 = model.add_plane_surface(channel_loop2,
                                         holes=[channel_loop, v_loop])

plane_surface3 = model.add_plane_surface(channel_loop3)
    
s_surface = model.add_plane_surface(s_loop)
    
v_surface = model.add_plane_surface(v_loop)

model.synchronize()

volume_marker = 6

model.add_physical([plane_surface,s_surface], "earth")
model.add_physical([plane_surface2, v_surface], "air")
model.add_physical([plane_surface3], "body")
    
    # I am not using this yet later in the code
model.add_physical([channel_lines[0],
                    channel_lines[2],
                    channel_lines[3],
                    channel_lines2[0],
                    channel_lines2[1],
                    channel_lines2[2]], "Walls")

geometry.generate_mesh(dim=2)

if 'nopopup' not in sys.argv:
    gmsh.fltk.run()